In [8]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [77]:
df = pd.read_csv('./data/POS-data_Baechlihof.csv', sep=";", encoding="latin-1")
#df = pd.read_csv('./data/POS-data_Juckerhof.csv', sep=";", encoding="latin-1")

In [78]:
df.head()

,timestamp,timestamp_human,shop,transaction_id,cost_unit,position_id,article,count,unit,article_group0,article_group1,article_group2,vat_pos,price
0,1485957427,01.02.2017 13:57:07,Juckerhof Bächlihof,2638192,400,7194841,Buffet - Dessert M,0.06,KG,HofKonditorei,HofKonditorei>>Dessert,NaN,0.15,2.05
1,1485963471,01.02.2017 15:37:51,Juckerhof Bächlihof,2638193,400,7194842,Kaffee,2.00,,Getränke,Getränke>>Heissgetränke,NaN,0.56,7.60
2,1485964029,01.02.2017 15:47:09,Juckerhof Bächlihof,2638194,400,7194843,Apfel-Birnen 3.50,2.40,KG,Obst,Obst>>Äpfel/Birnen,NaN,0.20,8.40
3,1485964084,01.02.2017 15:48:04,Juckerhof Bächlihof,2638195,400,7194844,Most past. 1 Liter,1.00,L,Getränke,Getränke>>Hoftee/Most,NaN,0.14,5.90
4,1485964248,01.02.2017 15:50:48,Juckerhof Bächlihof,2638196,400,7194845,Brot vom Hof,1.00,STK,HofBäckerei,HofBäckerei>>Brot,NaN,0.12,4.90


Average number of articles per basket

In [79]:
df.groupby('transaction_id').article.count().mean()

2.7108093018739683

Average basket price

In [80]:
df.groupby('transaction_id').price.mean().mean()

8.5324999605754783

Number of categories 

In [92]:
df.article_group0.unique()

array(['HofKonditorei', 'Getränke', 'Obst', 'HofBäckerei',
       'Frischprodukte', 'Gemüse', 'Trockenprodukte', 'Non-Food',
       'HofChuchi', 'Erlebnis'], dtype=object)

In [90]:
len(df.article_group0.unique()), len(df.article_group1.unique()), len(df.article_group2.unique())

(10, 49, 3)

In [91]:
df.article_group2.value_counts()

Fremd    118933
Eigen     60201
Name: article_group2, dtype: int64

Top 10 most sold products

In [81]:
df_count = df.groupby('article').count()[['count']]
df_count.sort_values(by="count", ascending=False).iloc[:10]

,count
article,
"Süssmost 1l, frisch gepresst",18479
PET Flasche,18251
Kuchen-/Gebäckstück 4 CHF,14492
Wähen süss,12412
Cappuccino,11814
Buffet - Zmittag Salat/Warm L,11788
Glace Kübeli,10206
Kaffee,10068
Apfelschorli 5dl,9914


# Trying to find communities of articles that work together

First, group by transaction_id all the articles under a list (consider this as a tokenized document)

In [87]:
df_transactions_full = df.groupby('transaction_id')['article'].apply(list)

Remove the transactions with only one article ? 

In [85]:
len(df_transactions)

164784

Take a random subset

In [169]:
N_RANDOM = 20000
df_transactions = df_transactions_full.sample(frac=1).iloc[:N_RANDOM]

In [170]:
from gensim.models.ldamodel import LdaModel
from gensim import corpora

In [171]:
dictionary = corpora.Dictionary(df_transactions.values)
dictionary.save('./dictionary.dict')

In [172]:
doc_term_matrix = [dictionary.doc2bow(text) for text in df_transactions.values]
corpora.MmCorpus.serialize('./corpus.mm', doc_term_matrix)

In [173]:
lda = LdaModel(corpus, id2word=dictionary, num_topics=100)

/anaconda3/envs/dlab/lib/python3.6/site-packages/gensim/models/ldamodel.py:802: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [174]:
print(lda.print_topics(num_topics=2, num_words=4))

[(55, '0.673*"Kaffee / Espresso" + 0.048*"Zitrone normal" + 0.044*"Glühwein" + 0.039*"Rüebli"'), (20, '0.302*"Balsamico weiss 5dl" + 0.245*"Baumnussen Frankreich " + 0.092*"Flamm kuchen Vom Garte" + 0.053*"HofChuchi Spezialpreis"')]


In [175]:
lda.save('./lda.model')
from gensim.models import LdaModel
loading = LdaModel.load('./lda.model')

Visualization

In [176]:
import pyLDAvis.gensim
import gensim
import operator
pyLDAvis.enable_notebook()

In [177]:
d = gensim.corpora.Dictionary.load('dictionary.dict')
c = gensim.corpora.MmCorpus('corpus.mm')
lda = gensim.models.LdaModel.load('lda.model')

From one or more articles, we want to find the next one that'd be nice to sell with.

<u> Method 1 : Handcrafting </u>

In [187]:
input_article = "Auberginen"
best_topics = dict(lda.get_term_topics(input_article, minimum_probability=.01))
sorted_best_topics = sorted(best_topics.items(), key=operator.itemgetter(1), reverse=True)
print("best topics:\n")
print(sorted_best_topics)
best_community_idx = lda.get_topic_terms(sorted_best_topics[0][0])
print([(dictionary.get(i), p) for (i,p) in best_community_idx])

best topics:

[(85, 0.15216638), (58, 0.035415795), (16, 0.030991629)]
[('Kaffee', 0.42478865), ('Auberginen', 0.15411475), ('HofChuchi Spezialpreis', 0.098582804), ('Peterli', 0.080427818), ('Gurke Nostrano', 0.043230966), ('Kuchenstück 4.-', 0.038418077), ('Trockenfleisch Eichenberger', 0.018223355), ('Kopfsalat', 0.016675303), ('Buffet - Zmittag Salat/Warm M', 0.011449457), ('Zierkürbismix in Kartonschale', 0.010535863)]


In [128]:
def handcraft_similarity(input_article):
    input_article = "Kaffee"
    best_topics = dict(lda.get_term_topics(input_article, minimum_probability=.2))
    sorted_best_topics = sorted(best_topics.items(), key=operator.itemgetter(1), reverse=True)
    best_community_idx = lda.get_topic_terms(sorted_best_topics[0][0])
    return [(dictionary.get(i), p) for (i,p) in best_community_idx]

<u> Similarity using topic embeddings </u>

In [129]:
def get_similar(date, articles_list, n_similar):
    """ from an articles list, find the n_similar other articles that'd
    best fit to be sold with it"""

In [185]:
dict(lda.get_term_topics("Kaffee", minimum_probability=.2))

{3: 0.23025934, 4: 0.2658352, 85: 0.42346159}

In [179]:
topics_terms = lda.state.get_lambda()
topics_terms_proba = np.apply_along_axis(lambda x: x/x.sum(),1,topics_terms)

In [182]:
topics_terms_proba.shape

(100, 884)

In [180]:
sum([p for (_,p) in lda.get_term_topics("Brötli vom Hof", minimum_probability=0.)])

ValueError: too many values to unpack (expected 2)

In [188]:
text = ["Brötli vom Hof"]
bow = dictionary.doc2bow(text)
lda.get_document_topics(bow, minimum_probability=0, minimum_phi_value=0);

In [189]:
dictionary.get(216)

'Quöllfrisch 3.3dl'

Autre méthode : prendre l'article dont l'embedding est le plus proche

In [190]:
data = pyLDAvis.gensim.prepare(lda, c, d)
data

/anaconda3/envs/dlab/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
38     7.552772        1       1 -0.017004  0.141647
90     4.469117        1       2 -0.069055 -0.025036
69     3.374006        1       3 -0.065700 -0.010420
92     3.032253        1       4  0.189310 -0.054302
93     2.744361        1       5 -0.116442 -0.039971
54     2.198153        1       6 -0.141558 -0.002441
63     2.147914        1       7 -0.013121 -0.040210
83     2.080435        1       8  0.104470 -0.024584
33     1.814672        1       9 -0.083221 -0.024553
55     1.791233        1      10  0.064679 -0.001358
13     1.707691        1      11  0.044659  0.000426
77     1.671558        1      12 -0.018628 -0.096356
25     1.638744        1      13  0.261226 -0.004086
91     1.533592        1      14 -0.077029 -0.022071
66     1.482303        1      15 -0.185981 -0.069313
8      1.465643        1      16  0.316922 -0.180249
19     1.408697        1      17  0.058024  0.179713
85     1.403152        1      18  0.085345  0.241025
75     1.371526        1      19 -0.065092  0.030338
37     1.361842        1      20 -0.087574 -0.092121
7      1.314907        1      21  0.095068  0.103298
61     1.267887        1      22  0.116491  0.222936
10     1.225010        1      23  0.071540 -0.128938
31     1.174188        1      24 -0.113318  0.012051
60     1.156716        1      25 -0.134821 -0.017741
3      1.140903        1      26  0.139133  0.196103
6      1.120515        1      27 -0.153465 -0.078968
36     1.120418        1      28 -0.092755 -0.089008
51     1.098404        1      29 -0.106507 -0.043768
21     1.065526        1      30 -0.100221  0.044180
...         ...      ...     ...       ...       ...
20     0.489693        1      71 -0.027631  0.012623
76     0.488628        1      72  0.116753  0.105397
28     0.477647        1      73 -0.022689 -0.039370
46     0.471317        1      74 -0.102625  0.036224
42     0.470698        1      75  0.008425 -0.128962
78     0.466568        1      76  0.262747 -0.240298
47     0.453272        1      77 -0.006596  0.005351
88     0.437391        1      78 -0.081252 -0.094160
89     0.436572        1      79 -0.041504 -0.065545
40     0.436351        1      80 -0.051576  0.089044
44     0.435630        1      81  0.011110 -0.145656
22     0.428394        1      82 -0.050598  0.004315
45     0.423140        1      83 -0.138667 -0.047173
98     0.414982        1      84 -0.104947 -0.107838
1      0.408374        1      85 -0.063031 -0.076930
80     0.405714        1      86  0.148034  0.036859
30     0.401413        1      87 -0.112740 -0.043090
59     0.386542        1      88 -0.046370  0.090856
34     0.384421        1      89 -0.168651 -0.037086
48     0.375075        1      90  0.092761 -0.091871
56     0.353125        1      91  0.149510  0.085571
32     0.336282        1      92 -0.004769  0.186505
70     0.327392        1      93  0.186771 -0.064485
68     0.317341        1      94 -0.067289 -0.037190
14     0.314981        1      95  0.090532  0.093707
95     0.300632        1      96  0.008504  0.153254
2      0.299018        1      97  0.245315 -0.134869
65     0.283431        1      98  0.065261 -0.183065
35     0.280965        1      99 -0.062185 -0.021654
97     0.275196        1     100 -0.023270  0.066377

[100 rows x 5 columns], topic_info=      Category         Freq                                   Term  \
term                                                                 
57     Default  2972.000000                 HofChuchi Spezialpreis   
7      Default  2234.000000              Kuchen-/Gebäckstück 4 CHF   
50     Default  1451.000000                                 Kaffee   
19     Default   974.000000                      Kaffee / Espresso   
34     Default  1290.000000                            Flammkuchen   
35     Default  1291.000000                  Apfelringe Jucker 90g   
6      Default  1322.000000       